In [ ]:
"""

Module: WorldPop Population Data Processing Script

Script Functionality:
This script processes WorldPop population data for specific countries and regions using Google Earth Engine (GEE). 
The script loads a WorldPop ImageCollection, filters it for target countries, merges the population data, 
and reprojects the results to a desired CRS and resolution. It then resamples the population data to a lower resolution 
(500m), clips it to a specified region of interest (ROI), and exports the resulting GeoTIFF files for each population band.

Input Parameters:
WorldPop ImageCollection (WorldPop/GP/100m/pop_age_sex).
A shapefile representing the region of interest (ROI).

Dependencies:
Earth Engine API : pip install earthengine-api (for accessing GEE data and functions)
Geemap : pip install geemap (for visualizing and interacting with GEE in Python)

Version Information:
Version: v1.0
Change Log: October 09 2024.
"""
import ee
import geemap
# Initialize the Earth Engine API
ee.Authenticate()
ee.Initialize()

# 1. Load WorldPop ImageCollection
worldpop_collection = ee.ImageCollection("WorldPop/GP/100m/pop_age_sex")

# Print the number of images in the collection to confirm successful loading
print('Number of images in WorldPop ImageCollection:', worldpop_collection.size().getInfo())

# 2. Load ROI (Region of Interest) table (Shapefile)
roi = ee.FeatureCollection("projects/ee-meiyang150269/assets/boundary")  # Replace with your Shapefile asset path
Map = geemap.Map()
# Optionally display the ROI on the map
Map.centerObject(roi, 7)  # Adjust zoom level as needed
Map.addLayer(roi, {'color': 'red'}, 'ROI')

# 3. Define the country names of interest
target_countries = ['BFA', 'TGO', 'GHA', 'CIV']

# 4. Filter the ImageCollection for images covering the target countries
filtered_collection = worldpop_collection.filter(ee.Filter.inList('country', target_countries))

# Print the number of images after filtering
print('Number of images in the filtered ImageCollection:', filtered_collection.size().getInfo())

# 5. Merge the filtered images into a single image
merged_image = filtered_collection.mosaic()

# Print the band names of the merged image
print('Band names of the merged image:', merged_image.bandNames().getInfo())

# 6. Define the target projection (using EPSG:3857)
crs = 'EPSG:3857'
scale_100m = 100  # 100-meter resolution
scale_500m = 500  # 500-meter resolution
#scale_1000m = 1000
# 7. Reproject to EPSG:3857 with 100m resolution
population_100m = merged_image.reproject(crs=crs, scale=scale_100m)

# 8. Check the data properties
sample = population_100m.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=roi,
    scale=scale_100m,
    maxPixels=1e10
)
print('Sample population value:', sample.getInfo())

# Define the pixel area for a 100m raster in square meters
pixel_area_100m = ee.Image.pixelArea().reproject(crs=crs, scale=scale_100m)
population_counts = population_100m.multiply(pixel_area_100m).rename(merged_image.bandNames())

# 9. Resample the 100m population data to 500m, summing the population in each 500m grid
population_500m_sum = population_counts.reduceResolution(
    reducer=ee.Reducer.sum(),
    maxPixels=500#1000
).reproject(crs=crs, scale=scale_500m)#1000

# 10. Clip to the ROI
final_image = population_500m_sum.clip(roi)

# Print the band names of the clipped image
print('Band names of the clipped image:', final_image.bandNames().getInfo())

# 11. Export the resampled result for each band
band_names = final_image.bandNames().getInfo()

for band in band_names:
    task = ee.batch.Export.image.toDrive(
        image=final_image.select(band),
        description='WorldPop_500m_' + band + '_Total_Population',
        #description='WorldPop_1000m_' + band + '_Total_Population',
        folder='WorldPop_500m_Exports',  # Specify the folder in Google Drive
        #folder='WorldPop_1000m_Exports',
        fileNamePrefix='WorldPop_500m_' + band,
        #fileNamePrefix='WorldPop_1000m_' + band,
        crs=crs,
        scale=scale_500m,  # 500-meter resolution
        #scale=scale_1000m, 
        region=roi.geometry(),  # Export region
        fileFormat='GeoTIFF',
        maxPixels=1e13
    )
    task.start()

# End of the script

resample

In [ ]:
"""

Module: WorldPop Raster Resampling Script

Script Functionality:
This script resamples WorldPop population data (or other raster datasets) based on a high-resolution reference raster. 
It reads raster files from an input folder, reprojects and resamples each file to match the resolution and projection 
of a specified reference raster, and outputs the resampled files to a designated folder.

Input Parameters:
A reference raster file (.tif) with the desired target resolution and projection.
A folder containing input raster files (.tif) that need to be resampled.

Dependencies:
GDAL : pip install gdal (for raster processing and reprojection)
Rasterio : pip install rasterio (optional, for additional raster operations)
NumPy : pip install numpy (for numerical array operations if needed)


Version Information:
Version: v1.0
Change Log: October 09 2024.
"""
from osgeo import gdal
import rasterio
from rasterio.enums import Resampling
import os
import numpy as np

referencefilepath = r'D:\ghana\0.005.tif'
inputfolderpath = r'D:\ghana\worldpop\pop_age_sex\0.005'
inputfilepaths = []
outputfolderpath = r'D:\ghana\worldpop\pop_age_sex\0.005\Resample'
outputfilepaths = []

# Get projection information
referencefile = gdal.Open(referencefilepath, gdal.GA_ReadOnly)
rfproj = referencefile.GetProjection()
bandreferencefile = referencefile.GetRasterBand(1)
rfTrans = referencefile.GetGeoTransform()
rfwidth = referencefile.RasterXSize
rfheight = referencefile.RasterYSize
nbands = referencefile.RasterCount

# Read data
file_list = os.listdir(inputfolderpath)
for filename in file_list:
    if filename.endswith('.tif'):
        inputpath = os.path.join(inputfolderpath, filename)
        outputpath = os.path.join(outputfolderpath, '0.005_' + filename)
        inputfilepaths.append(inputpath)
        outputfilepaths.append(outputpath)

print(inputfilepaths)

# Resampling
def ReprojectImages(inputFilePaths, outputFilePaths):
    for i in range(len(inputFilePaths)):
        inputfilePath = inputFilePaths[i]
        outputfilePath = outputFilePaths[i]
        # Get input raster file information
        inputrasfile = gdal.Open(inputfilePath, gdal.GA_ReadOnly)
        inputProj = inputrasfile.GetProjection()
        # Create resampled output file (set projection and six parameters)
        driver = gdal.GetDriverByName('GTiff')
        output = driver.Create(outputfilePath, rfwidth, rfheight, nbands, bandreferencefile.DataType)
        output.SetGeoTransform(rfTrans)
        output.SetProjection(rfproj)
        # Parameters: input dataset, output file, input projection, reference projection, resampling method (Nearest Neighbor, Bilinear, Cubic Convolution, etc.), callback function
        gdal.ReprojectImage(inputrasfile, output, inputProj, rfproj, gdal.GRA_NearestNeighbour)

ReprojectImages(inputfilepaths, outputfilepaths)

# End of the script

In [ ]:
"""

Module: Raster Data Resampling Script

Script Functionality:
This script resamples raster data files (.tif) located in a specified input folder. 
It processes each raster file to reduce its resolution by summing values based on a defined scale factor (5), 
resulting in a new resolution of 0.05. The resampled raster files are saved in a designated output folder, 
with filenames updated to reflect the new resolution.

Input Parameters:
input_folder: A folder containing raster files (.tif) to be resampled.
output_folder: A folder where the resampled raster files will be saved.

Dependencies:
GDAL: pip install gdal (for raster processing and reprojection)
Rasterio: pip install rasterio (for raster file operations)
NumPy: pip install numpy (for numerical array operations)

Version Information:
Version: v1.0
Change Log: November 03, 2024.
"""
import os
import rasterio
from rasterio.enums import Resampling
import numpy as np

# Set input and output folder paths
input_folder = r"D:\ghana\11.3\age and sex structures"  # Replace with your input folder path
output_folder = r"D:\ghana\11.3\age and sex structures1"  # Replace with your output folder path

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process all TIFF files in batch
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):
        input_file = os.path.join(input_folder, filename)  # Construct full path
        print(f"Processing file: {input_file}")
        
        with rasterio.open(input_file) as src:
            data = src.read(1)  # Read the first band
            profile = src.profile
            
            # Set new resolution (0.05) and calculate scale factor
            scale_factor = 5  # Aggregation factor from 0.01 to 0.05
            new_height = data.shape[0] // scale_factor
            new_width = data.shape[1] // scale_factor

            # Sum aggregation resampling
            resampled_data = data[:new_height * scale_factor, :new_width * scale_factor].reshape(
                new_height, scale_factor, new_width, scale_factor).sum(axis=(1, 3))

            # Update profile
            profile.update({
                "height": new_height,
                "width": new_width,
                "transform": src.transform * src.transform.scale(scale_factor, scale_factor)
            })

        # Generate new output filename
        new_filename = filename.replace("0.01", "0.05")  # Replace "0.01" in the filename with "0.05"
        output_file = os.path.join(output_folder, new_filename)

        # Export the result
        with rasterio.open(output_file, "w", **profile) as dst:
            dst.write(resampled_data, 1)

        print(f"File saved to: {output_file}")

print("All files have been processed.")

# End of the script

All files have been processed.
